In [1]:
# 1. Imports y Configuración
import sys
from pathlib import Path
import json

# Ajustar la ruta para que Python reconozca 'src'
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from src.config import MIDI_DIR, WAV_DIR, JAMS_DIR, DEFAULT_TEMPO
from src.generate_progression import generate_progression
from src.audio_conversion import convert_all_mid_in_folder
from src.jams_creation import create_jams_file

print("MIDI_DIR =", MIDI_DIR)
print("WAV_DIR  =", WAV_DIR)
print("JAMS_DIR =", JAMS_DIR)
print("DEFAULT_TEMPO =", DEFAULT_TEMPO)

MIDI_DIR = /home/cepatinog/MIR/final_project/chord_progression_generator/data/midi
WAV_DIR  = /home/cepatinog/MIR/final_project/chord_progression_generator/data/wav
JAMS_DIR = /home/cepatinog/MIR/final_project/chord_progression_generator/data/jams
DEFAULT_TEMPO = 60


In [2]:
# 2. Generar progresiones .mid
prog_all_1 = "I-IV-V"
name_all_1 = prog_all_1

generate_progression(prog_all_1, name_all_1, MIDI_DIR, DEFAULT_TEMPO)

midi_folder_all_1 = MIDI_DIR / name_all_1

Generated progression 'I-IV-V' -> folder: /home/cepatinog/MIR/final_project/chord_progression_generator/data/midi/I-IV-V


In [3]:
# 3. Crear .jams parseando nombre del .mid y leyendo durations.json

def strip_suffix(token: str) -> str:
    if "_" in token:
        return token.split("_", 1)[0]
    return token

def parse_filename_and_create_jams(midi_file: Path, durations_dict: dict):
    base_name = midi_file.stem
    parts = base_name.split("-")
    if len(parts) < 3:
        print(f"Nombre de archivo inesperado => {base_name}")
        return

    note_plus_oct = "-".join(parts[:2])
    progression_list = parts[2:-1]
    raw_tokens = "-".join(progression_list).split("-")
    roman_sequence = [strip_suffix(tok) for tok in raw_tokens]

    splitted = note_plus_oct.split("-")
    tonalidad = strip_suffix(splitted[0]) if len(splitted) >= 1 else "C"

    durations = durations_dict.get(f"{base_name}.mid", None)

    jam_path = create_jams_file(
        roman_sequence=roman_sequence,
        key=tonalidad,
        jam_name=base_name,
        progression_name="-".join(roman_sequence),
        durations=durations
    )
    print(f"[JAMS] {midi_file.name} => {jam_path.name}")

def parse_and_create_jams_in_folder(folder: Path):
    if not folder.exists():
        print(f"No existe carpeta => {folder}")
        return

    mid_files = list(folder.rglob("*.mid"))
    if not mid_files:
        print(f"No hay .mid en => {folder}")
        return

    durations_path = folder / "durations.json"
    if durations_path.exists():
        with open(durations_path, "r") as f:
            durations_dict = json.load(f)
    else:
        durations_dict = {}

    for mf in mid_files:
        parse_filename_and_create_jams(mf, durations_dict)

In [4]:
# Ejecutar creación de .jams
parse_and_create_jams_in_folder(midi_folder_all_1)


[JAMS] G-4-I-IV-V-49.mid => G-4-I-IV-V-49.jams
[JAMS] F#-3-I-IV-V-16.mid => F#-3-I-IV-V-16.jams
[JAMS] C#-5-I-IV-V-15.mid => C#-5-I-IV-V-15.jams
[JAMS] D-4-I-IV-V-32.mid => D-4-I-IV-V-32.jams
[JAMS] D-4-I-IV-V-63.mid => D-4-I-IV-V-63.jams
[JAMS] Bb-3-I-IV-V-25.mid => Bb-3-I-IV-V-25.jams
[JAMS] F-4-I-IV-V-13.mid => F-4-I-IV-V-13.jams
[JAMS] C-5-I-IV-V-26.mid => C-5-I-IV-V-26.jams
[JAMS] C#-6-I-IV-V-30.mid => C#-6-I-IV-V-30.jams
[JAMS] C#-6-I-IV-V-29.mid => C#-6-I-IV-V-29.jams
[JAMS] Bb-5-I-IV-V-55.mid => Bb-5-I-IV-V-55.jams
[JAMS] E-5-I-IV-V-19.mid => E-5-I-IV-V-19.jams
[JAMS] Ab-2-I-IV-V-11.mid => Ab-2-I-IV-V-11.jams
[JAMS] A-3-I-IV-V-22.mid => A-3-I-IV-V-22.jams
[JAMS] Bb-2-I-IV-V-30.mid => Bb-2-I-IV-V-30.jams
[JAMS] Ab-3-I-IV-V-51.mid => Ab-3-I-IV-V-51.jams
[JAMS] G-5-I-IV-V-43.mid => G-5-I-IV-V-43.jams
[JAMS] B-6-I-IV-V-58.mid => B-6-I-IV-V-58.jams
[JAMS] F#-6-I-IV-V-12.mid => F#-6-I-IV-V-12.jams
[JAMS] C-6-I-IV-V-5.mid => C-6-I-IV-V-5.jams
[JAMS] G-4-I-IV-V-43.mid => G-4-I-IV-V-43.